In [1]:
!pip install datasets torchaudio transformers sentencepiece gTTS huggingface_hub accelerate unsloth git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-scbvslsj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-scbvslsj
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
!pip install huggingface_hub

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset
import torchaudio

dataset = load_dataset("google/fleurs", "en_us", split="train")
mini_fleurs = dataset.select(range(100))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

fleurs.py:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.tar.gz:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

...

dev.tsv:   0%|          | 0.00/213k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/368k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

model_name = "aditya168/finetune_GPT"

model1, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)

def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model1.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            no_repeat_ngram_size=2
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        embeddings = model1.get_input_embeddings()(inputs.input_ids).squeeze(0)
    return embeddings




🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
import torch
import whisper
import torchaudio
from whisper.audio import pad_or_trim

whisper_model = whisper.load_model("base")

def get_whisper_embeddings_from_fleurs(audio_sample):

    audio = torch.tensor(audio_sample['array'], dtype=torch.float32)

    if audio_sample['sampling_rate'] != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=audio_sample['sampling_rate'], new_freq=16000)
        audio = resampler(audio)

    if audio.ndim > 1:
        audio = audio.mean(dim=0)

    audio = pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).unsqueeze(0).to(whisper_model.device)

    with torch.no_grad():
        embeddings = whisper_model.encoder(mel).squeeze(0)

    return embeddings


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 72.3MiB/s]


In [7]:
import torch.nn as nn

class AudioAdapter(nn.Module):
    def __init__(self, input_dim=512, output_dim=3072):
        super().__init__()
        self.adapter = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim)
        )

    def forward(self, x):
        return self.adapter(x)



In [8]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

adapter = AudioAdapter().to("cuda")
optimizer = torch.optim.AdamW(adapter.parameters(), lr=1e-4)
loss_fn = nn.MSELoss(reduction="none")

def collate(batch):
    audio_embs, text_embs = [], []
    for example in batch:
        try:
            w = get_whisper_embeddings_from_fleurs(example["audio"])
            t = get_embeddings(example["transcription"])
            min_len = min(w.shape[0], t.shape[0])
            audio_embs.append(w[:min_len])
            text_embs.append(t[:min_len])
        except Exception:
            continue

    if not audio_embs:
        return None, None, None

    audio_padded = pad_sequence(audio_embs, batch_first=True)
    text_padded = pad_sequence(text_embs, batch_first=True)

    lengths = [a.shape[0] for a in audio_embs]
    mask = torch.zeros_like(text_padded[..., 0])
    for i, l in enumerate(lengths):
        mask[i, :l] = 1.0

    return audio_padded.to("cuda"), text_padded.to("cuda"), mask.to("cuda")

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate)

for epoch in range(5):
    for audio_embs, text_embs, mask in dataloader:
        if audio_embs is None:
            continue
        audio_embs = audio_embs.float()
        text_embs = text_embs.float()
        pred = adapter(audio_embs)
        loss_raw = loss_fn(pred, text_embs).mean(dim=-1)
        loss = (loss_raw * mask).sum() / mask.sum()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")


Epoch 1 Loss: 0.0006
Epoch 2 Loss: 0.0004
Epoch 3 Loss: 0.0004
Epoch 4 Loss: 0.0005
Epoch 5 Loss: 0.0004


In [9]:
import torch
import whisper
from whisper.audio import pad_or_trim, log_mel_spectrogram
import torchaudio

def preprocess_audio(audio_path, model):
    audio, sr = torchaudio.load(audio_path)
    if sr != 16000:
        audio = torchaudio.transforms.Resample(sr, 16000)(audio)
    if audio.ndim > 1:
        audio = audio.mean(dim=0)
    audio = pad_or_trim(audio)
    mel = log_mel_spectrogram(audio).unsqueeze(0).to(model.device)
    with torch.no_grad():
        embeddings = model.encoder(mel).squeeze(0)
    return embeddings

@torch.no_grad()
def generate_response_from_audio(audio_path):
    whisper_embeddings = preprocess_audio(audio_path, whisper_model)

    adapted_embeddings = adapter(whisper_embeddings).unsqueeze(0)

    dummy_input_ids = torch.tensor([[tokenizer.bos_token_id]]).to(adapted_embeddings.device)

    outputs = model1(
        input_ids=dummy_input_ids,
        inputs_embeds=adapted_embeddings,
        use_cache=True
    )

    logits = outputs.logits
    next_token = torch.argmax(logits[:, -1, :], dim=-1)
    generated_ids = [next_token.item()]

    for _ in range(100):
        input_ids = torch.tensor([generated_ids[-1:]]).to(adapted_embeddings.device)
        outputs = model1(input_ids=input_ids, use_cache=True)
        next_token = torch.argmax(outputs.logits[:, -1, :], dim=-1)
        generated_ids.append(next_token.item())
        if next_token.item() == tokenizer.eos_token_id:
            break

    response = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return response


In [13]:
audio_path = "/content/gtts_sample_output.wav"
whisper_emb = preprocess_audio(audio_path, whisper_model)

In [17]:
with torch.no_grad():
    adapted_emb = adapter(whisper_emb.to("cuda"))  # [T, 4096]
    adapted_emb = adapted_emb.to(dtype=token_embeds.dtype, device=token_embeds.device)

In [18]:
def embed_to_token_ids(embed_seq, token_embeddings):
    similarities = torch.matmul(embed_seq, token_embeddings.T)
    token_ids = similarities.argmax(dim=-1)
    return token_ids

result = whisper_model.transcribe(audio_path)

In [19]:
token_embeds = model1.model.model.embed_tokens.weight.detach()
recovered_token_ids = embed_to_token_ids(adapted_emb, token_embeds)
recovered_token_ids = recovered_token_ids.unsqueeze(0)
pseudo_inputs = {
    "input_ids": recovered_token_ids.clone(),
    "attention_mask": (recovered_token_ids != tokenizer.pad_token_id).long()
}
messages = [
    {"role": "system", "content": "You are an AI assistant. Your role is to give to the point answers. You need not to answer in more than 100 words"},
    {"role": "user", "content": result}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False)
true_inputs = tokenizer(prompt, return_tensors="pt").to(device)
if recovered_token_ids.shape[-1] < true_inputs["input_ids"].shape[-1]:
    pad_len = true_inputs["input_ids"].shape[-1] - recovered_token_ids.shape[-1]
    padded_token_ids = torch.nn.functional.pad(recovered_token_ids, (0, pad_len), value=tokenizer.pad_token_id)
else:
    padded_token_ids = recovered_token_ids[:, :true_inputs["input_ids"].shape[-1]]
outputs = model1.generate(
    input_ids=true_inputs["input_ids"],
    attention_mask=true_inputs["attention_mask"],
    max_new_tokens=170,
    do_sample=True,
    temperature=0.3,
    top_k=30,
    top_p=0.85,
    no_repeat_ngram_size=3
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.split("assistant")[-1].strip()


In [20]:
from gtts import gTTS
tts = gTTS(text=response, lang='en')

output_path = "test_audio.mp3"
tts.save(output_path)

print(f"Audio saved to: {output_path}")

Audio saved to: test_audio.mp3
